In [ ]:
import cv2 as cv
import numpy as np

def triangulate_ball_position(P_left, P_right, pt_left, pt_right):
    """
    Triangulate the 3D coordinates of a detected ball given:
      - P_left: Projection matrix for the left camera (3x4)
      - P_right: Projection matrix for the right camera (3x4)
      - pt_left: 2D coordinates in the left image (x, y)
      - pt_right: 2D coordinates in the right image (x, y)
    Returns:
      A numpy array of 3D coordinates [x, y, z].
    """
    # Convert the 2D points into 2x1 arrays (float32)
    pts_left = np.array(pt_left, dtype=np.float32).reshape(2, 1)
    pts_right = np.array(pt_right, dtype=np.float32).reshape(2, 1)
    
    # Triangulate the 4D homogeneous coordinates
    point4D = cv.triangulatePoints(P_left, P_right, pts_left, pts_right)
    # Convert from homogeneous (4D) to Euclidean coordinates (3D)
    point3D = point4D[:3] / point4D[3]
    return point3D.flatten()

# ----- Example usage -----
# Assume pipeline_result is the dictionary returned by the pipeline function,
# which contains for each stereo pair the projection matrices needed for triangulation.
# For example:
# pipeline_result = {
#     "Stereo_Pair_1": {
#         "Projection_Left": P_left1,  # 3x4 matrix for left camera of pair 1
#         "Projection_Right": P_right1,  # 3x4 matrix for right camera of pair 1
#         "Rotation": R1,
#         "Translation": T1
#     },
#     "Stereo_Pair_2": {
#         "Projection_Left": P_left2,  # 3x4 matrix for left camera of pair 2
#         "Projection_Right": P_right2,  # 3x4 matrix for right camera of pair 2
#         "Rotation": R2,
#         "Translation": T2
#     }
# }

# Example 2D detections from YOLO (for a single frame per stereo pair)
# Coordinates (in pixels) from each camera's detection of the ball.
pt_left_pair1 = (100, 150)    # left camera of stereo pair 1
pt_right_pair1 = (110, 148)   # right camera of stereo pair 1

pt_left_pair2 = (200, 250)    # left camera of stereo pair 2
pt_right_pair2 = (210, 248)   # right camera of stereo pair 2

# Retrieve projection matrices from the calibration pipeline result
# (These matrices combine intrinsics and extrinsics.)
P_left_pair1 = pipeline_result["Stereo_Pair_1"]["Projection_Left"]
P_right_pair1 = pipeline_result["Stereo_Pair_1"]["Projection_Right"]

P_left_pair2 = pipeline_result["Stereo_Pair_2"]["Projection_Left"]
P_right_pair2 = pipeline_result["Stereo_Pair_2"]["Projection_Right"]

# Triangulate the ball position for each stereo pair
ball_pos_pair1 = triangulate_ball_position(P_left_pair1, P_right_pair1, pt_left_pair1, pt_right_pair1)
ball_pos_pair2 = triangulate_ball_position(P_left_pair2, P_right_pair2, pt_left_pair2, pt_right_pair2)

print("3D Ball Position from Stereo Pair 1:", ball_pos_pair1)
print("3D Ball Position from Stereo Pair 2:", ball_pos_pair2)

# Optionally, fuse or select between the two estimates as needed.
